In [1]:
from tkinter import *
import requests
#import json
import datetime
from datetime import date
from datetime import timedelta
import pandas as pd
import numpy as np
#import random
#import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing
#SimpleExpSmoothing, Holt
#from sklearn.metrics import mean_squared_error
from sklearn import neighbors
#preprocessing
#from sklearn import cross_validation

In [2]:
class WeatherGrabber:
    
    def knnCondition(self, cn_data):
        
        current_weather = [cn_data['avg_temp1'], cn_data['pressure1'], cn_data['humid1'], cn_data['precip1'], cn_data['cloud1'], cn_data['gust1']]
        next_weather = [cn_data['avg_temp2'], cn_data['pressure2'], cn_data['humid2'], cn_data['precip2'], cn_data['cloud2'], cn_data['gust2']]
        cn_list = [current_weather, next_weather]
        
        list1 = cn_list[0]
        list2 = cn_list[1]
        
        hist_condition_data = pd.read_csv(r'knndataset.csv')

        x = np.array(hist_condition_data.drop(['condition'],1))
        y = np.array(hist_condition_data['condition'])

        #x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.1)

        clf = neighbors.KNeighborsClassifier()
        clf.fit(x, y)
        #clf.fit(x_train, y_train)

        #accuracyy = clf.score(x_test, y_test)
        #print(accuracyy)

        example = np.array([list1, list2])
        pred = clf.predict(example)
        return pred
    
    
    def grabHistWeather(self, city):
        days1 = [] #list of the past week's dates
        today = date.today()
        for i in range(1,8):
            days1.append(today - timedelta(days=i))
        for i in range(0,7): #making the dates strings
            days1[i] = days1[i].strftime('%Y-%m-%d')
        days1.reverse()
        histurl = [] #url string for api call to get weather data of each day of the past week for the input city

        for i in range(0,7):
            histurl.append('http://api.apixu.com/v1/history.json?key=ba8c99559eb84eec9a482825191502&q='+city+'&dt='+days1[i])
        histjson = [] #weather data of each day of the past week for the input city
        for i in range(0,7):
            histjson.append(requests.get(histurl[i]).json())
            
        hist_pressure = []
        hist_temp = []
        hist_humidity = []
        hist_cloud = []
        hist_precip = []
        hist_gust = []

        for i in range(0,7):
            for j in range(0,24):
                hist_pressure.append(histjson[i]['forecast']['forecastday'][0]['hour'][j]['pressure_mb'])

        for i in range(0,7):
            for j in range(0,24):
                hist_temp.append(histjson[i]['forecast']['forecastday'][0]['hour'][j]['temp_c'])

        for i in range(0,7):
            for j in range(0,24):
                hist_humidity.append(histjson[i]['forecast']['forecastday'][0]['hour'][j]['humidity'])

        for i in range(0,7):
            for j in range(0,24):
                hist_cloud.append(histjson[i]['forecast']['forecastday'][0]['hour'][j]['cloud'])

        for i in range(0,7):
            for j in range(0,24):
                hist_precip.append(histjson[i]['forecast']['forecastday'][0]['hour'][j]['precip_mm'])
        
        for i in range(0,7):
            for j in range(0,24):
                hist_gust.append(histjson[i]['forecast']['forecastday'][0]['hour'][j]['gust_kph'])
        
        localtime = histjson[0]['location']['localtime'].split()
        localdate = localtime[0]
        localtime = localtime[1]
        localdate = localdate.split('-')
        localdate = localdate[2] + '-' + localdate[1] + '-' + localdate[0]
        location = histjson[0]['location']['name'] + ', ' + histjson[0]['location']['region']
        country = histjson[0]['location']['country']
        
        hist_dict = {'temp': hist_temp,
                     'pressure': hist_pressure,
                     'humidity': hist_humidity, 
                     'precipitation': hist_precip,
                     'cloud': hist_cloud,
                     'gust': hist_gust,
                     'localtime': localtime,
                     'localdate': localdate,
                     'location': location,
                     'country': country}
        
        hist_df = pd.DataFrame(hist_dict)
        return hist_df
    
    
    def hwTrainer(self, hist_df):
        
        ttrain = hist_df['temp'].values
        tpredictions = []
        htrain = hist_df['humidity'].values
        hpredictions = []
        
        tmodel_hw_fit = ExponentialSmoothing(ttrain, seasonal_periods=24, seasonal='add').fit() #use_boxcox=True 
        hmodel_hw_fit = ExponentialSmoothing(htrain, seasonal_periods=24, seasonal='add').fit()
        
        tpredictions = tmodel_hw_fit.forecast(48)
        hpredictions = hmodel_hw_fit.forecast(48)
        for i in range(0,hpredictions.size):
            if hpredictions[i] <= 0:
                hpredictions[i] = 5; 
                
        tpredictions1 = tpredictions[0:24]
        tpredictions2 = tpredictions[24:]

        hpredictions1 = hpredictions[0:24]
        hpredictions2 = hpredictions[24:]
        
        current_temp = round(tpredictions1[int(datetime.datetime.now().strftime('%H'))], 1)
        current_humid = round(int(hpredictions1[int(datetime.datetime.now().strftime('%H'))]))
        
        current_max_temp = round(np.amax(tpredictions1), 1)
        next_max_temp = round(np.amax(tpredictions2)+0.3, 1)
        
        current_min_temp = round(np.amin(tpredictions1), 1)
        next_min_temp = round(np.amin(tpredictions2)-0.4, 1)
        
        current_avg_temp = round(np.mean(tpredictions1), 1)
        next_avg_temp = round(np.mean(tpredictions2)+0.4, 1)
        
        current_avg_humid = round(np.mean(hpredictions1))
        next_avg_humid = round(np.mean(hpredictions2)+5)
    
        trained_data = {'current_temp': current_temp,
                        'current_humid': current_humid, 
                        'current_max_temp': current_max_temp,
                        'next_max_temp': next_max_temp,
                        'current_min_temp': current_min_temp,
                        'next_min_temp': next_min_temp,
                        'current_avg_temp': current_avg_temp,
                        'next_avg_temp': next_avg_temp, 
                        'current_avg_humid': current_avg_humid,
                        'next_avg_humid': next_avg_humid}
        return trained_data
    
    
    def origWeather(self, city):
        forurl = 'https://api.apixu.com/v1/forecast.json?key=ba8c99559eb84eec9a482825191502&q='+city+'&days=2'
        q = requests.get(forurl).json()

        cur_temp = q['current']['temp_c']     
        cur_humid = q['current']['humidity']

        max_temp1 = q['forecast']['forecastday'][0]['day']['maxtemp_c']
        min_temp1 = q['forecast']['forecastday'][0]['day']['mintemp_c']
        avg_temp1 = q['forecast']['forecastday'][0]['day']['avgtemp_c']
        pressure1 = q['current']['pressure_mb']
        humid1 = q['forecast']['forecastday'][0]['day']['avghumidity']
        precip1 = q['forecast']['forecastday'][0]['day']['totalprecip_mm']
        cloud1 = q['current']['cloud']
        gust1 = q['current']['gust_kph']
        cond1 = WeatherGrabber.condManager(q['forecast']['forecastday'][0]['day']['condition']['text'])

        max_temp2 = q['forecast']['forecastday'][1]['day']['maxtemp_c']
        min_temp2 = q['forecast']['forecastday'][1]['day']['mintemp_c']
        avg_temp2 = q['forecast']['forecastday'][1]['day']['avgtemp_c']
        pressure2 = q['current']['pressure_mb']
        humid2 = q['forecast']['forecastday'][1]['day']['avghumidity']
        precip2 = q['forecast']['forecastday'][1]['day']['totalprecip_mm']
        cloud2 = q['current']['cloud']
        gust2 = q['current']['gust_kph']
        cond2 = WeatherGrabber.condManager(q['forecast']['forecastday'][1]['day']['condition']['text'])

        orig_data = {'cur_temp': cur_temp,
                     'cur_humid': cur_humid,
                     'max_temp1': max_temp1, 
                     'min_temp1': min_temp1,
                     'avg_temp1': avg_temp1,
                     'pressure1': pressure1,
                     'humid1': humid1,
                     'precip1': precip1,
                     'cloud1': cloud1,
                     'gust1': gust1,
                     'cond1': cond1,
                     'max_temp2': max_temp2,
                     'min_temp2': min_temp2,
                     'avg_temp2': avg_temp2,
                     'pressure2' :pressure2,
                     'humid2': humid2,
                     'precip2': precip2,
                     'cloud2': cloud2,
                     'gust2': gust2,
                     'cond2': cond2}
        return orig_data
    
    
    def condManager(str1):
    
        if str1 == 'Sunny' or str1 == 'Partly cloudy':
            str1 = 'Clear'
        elif str1 == 'Moderate or heavy rain shower' or str1 == 'Light rain shower' or str1 == 'Moderate rain at times' or str1 == 'Patchy light rain' or str1 == 'Patchy light rain with thunder' or str1 == 'Moderate rain' or str1 == 'Drizzle' or str1 == 'Light drizzle' or str1 == 'Light rain' or str1 == 'Heavy rain' or str1 == 'Moderate or heavy rain with thunder' or str1 == 'Light freezing rain' or str1 == 'Torrential rain shower':
            str1 = 'Rainy'
        elif str1 == 'Heavy rain at times' or str1 == 'Patchy rain possible' or str1 == 'Cloudy' or str1 == 'Overcast' or str1 == 'Patchy light drizzle' or str1 == 'Thundery outbreaks possible':
            str1 = 'Chance of rain'
        elif str1 == 'Moderate snow' or str1 == 'Heavy snow' or str1 == 'Patchy moderate snow' or str1 == 'Light snow' or str1 == 'Blizzard' or str1 == 'Moderate or heavy snow showers' or str1 == 'Moderate or heavy sleet' or str1 == 'Light sleet' or str1 == 'Patchy heavy snow' or str1 == 'Light snow showers' or str1 == 'Moderate or heavy snow with thunder' or str1 == 'Patchy snow possible':
            str1 = 'Snowfall'
        elif str1 == 'Fog' or str1 == 'Mist':
            str1 = 'Fog'
        return str1

In [3]:
class TheGUI:
    
    def __init__(self, master):
        
        master.geometry('250x250')

        img = PhotoImage(file=r'b.png')
        img = img.subsample(1, 1)

        background = Label(master, image=img, bd=0)
        background.pack(fill='both', expand=True)
        background.image = img
        
        #master.attributes("-fullscreen", True)
        master.title("Weather Predictor")
        #master.state('zoomed')
        master.state('normal')
        
        rows = 0
        while rows < 50:
            master.rowconfigure(rows, weight=1)
            master.columnconfigure(rows,weight=1)
            rows += 1
            
        rows = 0
        while rows < 50:
            background.rowconfigure(rows, weight=1)
            background.columnconfigure(rows,weight=1)
            rows += 1
        
        font1 = ("Times New Roman", 40, "bold")
        
        self.button1 = Button(background, font = font1, text = 'predict weather', fg = 'white', bg = '#00ced1', activeforeground = 'white', activebackground = '#1e90ff', bd = 0, command = p_window)
        self.button2 = Button(background, font = font1, text = 'about', fg = 'white', bg = '#00ced1', bd = 0, activeforeground = 'white', activebackground = '#1e90ff', command = a_window)
        self.button3 = Button(background, font = font1, text = "quit", fg = 'white', bg = '#00ced1', bd = 0, activeforeground = 'white', activebackground = '#1e90ff', command = master.destroy)
        
        self.button1.grid(row = 18, column = 24)
        self.button2.grid(row = 25, column = 24)
        self.button3.grid(row = 32, column = 24)
        
        master.mainloop()
        
            
def p_window():
        
    global window1 #we make it global in order to be able to call it from the validate button inside texter() function
    window1 = Toplevel(root)
    #window1.state('zoomed')
    window1.state('normal')
    window1.configure(background="white")   
       
    rows = 0
    while rows < 50:
        window1.rowconfigure(rows, weight=1)
        window1.columnconfigure(rows,weight=1)
        rows += 1
            
    font1 = ("Times New Roman", 25, "bold")
    font2 = ("Times New Roman", 16)
    font3 = ("Times New Roman", 18)
    font4 = ("Times New Roman", 21, "bold")
     
    l1 = Label(window1, font = font1, text = 'Enter any city name', fg = '#1e90ff', bg = 'white')
    e1 = Entry(window1, font = font1, textvariable = txtin, bg = 'white')
    b1 = Button(window1, font = font3, text ="predict weather", fg = 'white', bg = '#00ced1', activeforeground = 'white', activebackground = '#1e90ff', bd = 0, command = texter)
    
    lo1v = Label(window1, font = font2, textvariable = location1, bg = 'white')
    lo2v = Label(window1, font = font2, textvariable = location2, bg = 'white')
    lo3v = Label(window1, font = font2, textvariable = location3, bg = 'white')
    lo4v = Label(window1, font = font2, textvariable = location4, bg = 'white')
    
    lo1 = Label(window1, font = font2, textvariable = location1v, fg = '#1e90ff', bg = 'white')
    lo2 = Label(window1, font = font2, textvariable = location2v, fg = '#1e90ff', bg = 'white')
    lo3 = Label(window1, font = font2, textvariable = location3v, fg = '#1e90ff', bg = 'white')
    lo4 = Label(window1, font = font2, textvariable = location4v, fg = '#1e90ff', bg = 'white')
    
    tod1v = Label(window1, font = font4, textvariable = today1v, fg = '#1e90ff', bg = 'white')
    tod2v = Label(window1, font = font2, textvariable = today2v, fg = '#1e90ff', bg = 'white')
    tod3v = Label(window1, font = font2, textvariable = today3v, fg = '#1e90ff', bg = 'white')
    tod4v = Label(window1, font = font2, textvariable = today4v, fg = '#1e90ff', bg = 'white')
    tod5v = Label(window1, font = font2, textvariable = today5v, fg = '#1e90ff', bg = 'white')
    tod6v = Label(window1, font = font2, textvariable = today6v, fg = '#1e90ff', bg = 'white')
    
    tod2 = Label(window1, font = font2, textvariable = today2, bg = 'white')
    tod3 = Label(window1, font = font2, textvariable = today3, bg = 'white')
    tod4 = Label(window1, font = font2, textvariable = today4, bg = 'white')
    tod5 = Label(window1, font = font2, textvariable = today5, bg = 'white')
    tod6 = Label(window1, font = font2, textvariable = today6, bg = 'white')
    
    tom1v = Label(window1, font = font4, textvariable = tomo1v, fg = '#1e90ff', bg = 'white')
    tom2v = Label(window1, font = font2, textvariable = tomo2v, fg = '#1e90ff', bg = 'white')
    tom3v = Label(window1, font = font2, textvariable = tomo3v, fg = '#1e90ff', bg = 'white')
    tom4v = Label(window1, font = font2, textvariable = tomo4v, fg = '#1e90ff', bg = 'white')
    tom5v = Label(window1, font = font2, textvariable = tomo5v, fg = '#1e90ff', bg = 'white')
    
    tom2 = Label(window1, font = font2, textvariable = tomo2, bg = 'white')
    tom3 = Label(window1, font = font2, textvariable = tomo3, bg = 'white')
    tom4 = Label(window1, font = font2, textvariable = tomo4, bg = 'white')
    tom5 = Label(window1, font = font2, textvariable = tomo5, bg = 'white')
    
    rtoday1v = Label(window1, font = font2, textvariable = rtod1v, fg = '#1e90ff', bg = 'white')
    rtoday2v = Label(window1, font = font2, textvariable = rtod2v, fg = '#1e90ff', bg = 'white')
    rtoday3v = Label(window1, font = font2, textvariable = rtod3v, fg = '#1e90ff', bg = 'white')
    rtoday4v = Label(window1, font = font2, textvariable = rtod4v, fg = '#1e90ff', bg = 'white')
    rtoday5v = Label(window1, font = font2, textvariable = rtod5v, fg = '#1e90ff', bg = 'white')
    
    rtoday1 = Label(window1, font = font2, textvariable = rtod1, bg = 'white')
    rtoday2 = Label(window1, font = font2, textvariable = rtod2, bg = 'white')
    rtoday3 = Label(window1, font = font2, textvariable = rtod3, bg = 'white')
    rtoday4 = Label(window1, font = font2, textvariable = rtod4, bg = 'white')
    rtoday5 = Label(window1, font = font2, textvariable = rtod5, bg = 'white')
    
    rtomo1v = Label(window1, font = font2, textvariable = rtom1v, fg = '#1e90ff', bg = 'white')
    rtomo2v = Label(window1, font = font2, textvariable = rtom2v, fg = '#1e90ff', bg = 'white')
    rtomo3v = Label(window1, font = font2, textvariable = rtom3v, fg = '#1e90ff', bg = 'white')
    rtomo4v = Label(window1, font = font2, textvariable = rtom4v, fg = '#1e90ff', bg = 'white')
    
    rtomo1 = Label(window1, font = font2, textvariable = rtom1, bg = 'white')
    rtomo2 = Label(window1, font = font2, textvariable = rtom2, bg = 'white')
    rtomo3 = Label(window1, font = font2, textvariable = rtom3, bg = 'white')
    rtomo4 = Label(window1, font = font2, textvariable = rtom4, bg = 'white')
    
    b2 = Button(window1, font = font1, text ="Go Back", fg = 'white', bg = '#00ced1', activeforeground = 'white', activebackground = '#1e90ff', bd = 0, command = window1.destroy)
        
    l1.grid(row = 3, column = 1, sticky = E)
    e1.grid(row = 3, column = 2)
    b1.grid(row = 3, column = 3, sticky = W)
    
    lo1.grid(row = 4, column = 1, sticky = E)
    lo2.grid(row = 5, column = 1, sticky = E)
    lo3.grid(row = 6, column = 1, sticky = E)
    lo4.grid(row = 7, column = 1, sticky = E)
    
    lo1v.grid(row = 4, column = 2, sticky = W)
    lo2v.grid(row = 5, column = 2, sticky = W)
    lo3v.grid(row = 6, column = 2, sticky = W)
    lo4v.grid(row = 7, column = 2, sticky = W)
    
    tod1v.grid(row = 8, column = 1, sticky = E)
    tod2v.grid(row = 9, column = 1, sticky = E)
    tod3v.grid(row = 10, column = 1, sticky = E)
    tod4v.grid(row = 11, column = 1, sticky = E)
    tod5v.grid(row = 12, column = 1, sticky = E)
    tod6v.grid(row = 13, column = 1, sticky = E)
    
    tod2.grid(row = 9, column = 2, sticky = W)
    tod3.grid(row = 10, column = 2, sticky = W)
    tod4.grid(row = 11, column = 2, sticky = W)
    tod5.grid(row = 12, column = 2, sticky = W)
    tod6.grid(row = 13, column = 2, sticky = W)
    
    tom1v.grid(row = 14, column = 1, sticky = E)
    tom2v.grid(row = 16, column = 1, sticky = E)
    tom3v.grid(row = 17, column = 1, sticky = E)
    tom4v.grid(row = 18, column = 1, sticky = E)  
    tom5v.grid(row = 19, column = 1, sticky = E)
    
    tom2.grid(row = 16, column = 2, sticky = W)
    tom3.grid(row = 17, column = 2, sticky = W)
    tom4.grid(row = 18, column = 2, sticky = W)  
    tom5.grid(row = 19, column = 2, sticky = W)
    
    rtoday1v.grid(row = 9, column = 3, sticky = E)
    rtoday2v.grid(row = 10, column = 3, sticky = E)
    rtoday3v.grid(row = 11, column = 3, sticky = E)
    rtoday4v.grid(row = 12, column = 3, sticky = E)
    rtoday5v.grid(row = 13, column = 3, sticky = E)
    
    rtoday1.grid(row = 9, column = 4, sticky = W)
    rtoday2.grid(row = 10, column = 4, sticky = W)
    rtoday3.grid(row = 11, column = 4, sticky = W)
    rtoday4.grid(row = 12, column = 4, sticky = W)
    rtoday5.grid(row = 13, column = 4, sticky = W)
    
    rtomo1v.grid(row = 16, column = 3, sticky = E)
    rtomo2v.grid(row = 17, column = 3, sticky = E)
    rtomo3v.grid(row = 18, column = 3, sticky = E) 
    rtomo4v.grid(row = 19, column = 3, sticky = E) 
    
    rtomo1.grid(row = 16, column = 4, sticky = W)
    rtomo2.grid(row = 17, column = 4, sticky = W)
    rtomo3.grid(row = 18, column = 4, sticky = W) 
    rtomo4.grid(row = 19, column = 4, sticky = W)    
    b2.grid(row = 45, column = 1, sticky = W)
        
def texter(): #this function is invoked when 'predict weather' button inside window1 is clicked
    
    txt = txtin.get()
    city = str(txt)  
    w_obj = WeatherGrabber()
    hist_df = w_obj.grabHistWeather(city)
    trained_data = w_obj.hwTrainer(hist_df)
    cn_data = w_obj.origWeather(city)
    predcond = w_obj.knnCondition(cn_data)
    
    font2 = ("Times New Roman", 13)
    font3 = ("Times New Roman", 22)
    
    location1v.set('Location: ')
    location2v.set('Country: ')
    location3v.set('Localtime: ')
    location4v.set('Date: ')

    location1.set(hist_df.loc[0, 'location'])
    location2.set(hist_df.loc[0, 'country'])
    location3.set(hist_df.loc[0, 'localtime'])
    location4.set(hist_df.loc[0, 'localdate'])
    
    today1v.set("Today's forecast:")
    today2v.set('Current temperature: ')
    today3v.set('Maximum temperature: ') 
    today4v.set('Minimum temperature: ') 
    today5v.set('Average temperature: ')
    today6v.set('Condition: ')
    
    today2.set(str(trained_data['current_temp']) + '°C')
    today3.set(str(trained_data['current_max_temp']) + '°C') 
    today4.set(str(trained_data['current_min_temp']) + '°C') 
    today5.set(str(trained_data['current_avg_temp']) + '°C')
    today6.set(predcond[0])   
    
    tomo1v.set("Tomorrow's forecast:")
    tomo2v.set('Maximum temperature: ') 
    tomo3v.set('Minimum temperature: ') 
    tomo4v.set('Average temperature: ')
    tomo5v.set('Condition: ')
    
    tomo2.set(str(trained_data['next_max_temp']) + '°C') 
    tomo3.set(str(trained_data['next_min_temp']) + '°C') 
    tomo4.set(str(trained_data['next_avg_temp']) + '°C')
    tomo5.set(predcond[1])
    
    rtod1v.set('')
    rtod2v.set('')
    rtod3v.set('')
    rtod4v.set('')
    rtod5v.set('') 
    
    rtod1.set('')
    rtod2.set('')
    rtod3.set('')
    rtod4.set('')
    rtod5.set('')  
    rtom1v.set('')
    rtom2v.set('')
    rtom3v.set('')
    rtom4v.set('') 
    rtom1.set('')
    rtom2.set('')
    rtom3.set('')
    rtom4.set('')
    
    b1 = Button(window1, font = font3, text = 'validate', fg = 'white', bg = '#00ced1', activeforeground = 'white', activebackground = '#1e90ff', bd = 0, command = texter2)
    b1.grid(row = 25, column = 2, sticky = W)
    
def texter2(): #this function is invoked when 'validate' button inside window1 is clicked

    txt = txtin.get()
    city = str(txt)  
    w_obj = WeatherGrabber()
    cn_data = w_obj.origWeather(city)
    
    rtod1v.set('Real current temperature: ')
    rtod2v.set('Real maximum temperature: ') 
    rtod3v.set('Real minimum temperature: ') 
    rtod4v.set('Real average temperature: ')
    rtod5v.set('Real condition: ')
    
    rtod1.set(str(cn_data['cur_temp']) + '°C')
    rtod2.set(str(cn_data['max_temp1']) + '°C') 
    rtod3.set(str(cn_data['min_temp1']) + '°C') 
    rtod4.set(str(cn_data['avg_temp1']) + '°C')
    rtod5.set(str(cn_data['cond1']))
    
    rtom1v.set('Real maximum temperature: ') 
    rtom2v.set('Real minimum temperature: ') 
    rtom3v.set('Real average temperature: ')
    rtom4v.set('Real condition: ')

    rtom1.set(str(cn_data['max_temp2']) + '°C') 
    rtom2.set(str(cn_data['min_temp2']) + '°C') 
    rtom3.set(str(cn_data['avg_temp2']) + '°C')
    rtom4.set(str(cn_data['cond2']))
    
def a_window():
            
    global window2
    window2 = Toplevel(root)
    #window2.state('zoomed')
    window2.state('normal')
    window2.configure(background="white")
    
    rows = 0
    while rows < 50:
        window2.rowconfigure(rows, weight=1)
        window2.columnconfigure(rows,weight=1)
        rows += 1
        
    font1 = ("Times New Roman", 25, "bold")
    l = Label(window2, text="Credits", font = font1, bg = '#00ced1', fg = 'white')
    l1 = Label(window2, text="Advisor:\nSumit Gupta Sir\n\nDeveloped By:\nSoumya Porel", font = font1, bg = "white", fg = '#1e90ff')
    b1 = Button(window2, text ="Go Back", font = font1, fg = 'white', bg = '#00ced1', activeforeground = 'white', activebackground = '#1e90ff', bd=0, command = window2.destroy)
    
    l.grid(row=10, column=5, sticky=W)
    l1.grid(row=22, column=23, sticky=W)
    b1.grid(row=40, column=5, sticky=W)
    
    
    
    
root = Tk()

txtin = StringVar()

location1v = StringVar()
location2v = StringVar()
location3v = StringVar()
location4v = StringVar()

location1 = StringVar()
location2 = StringVar()
location3 = StringVar()
location4 = StringVar()

today1v = StringVar()
today2v = StringVar()
today3v = StringVar()
today4v = StringVar()
today5v = StringVar()
today6v = StringVar()

today2 = StringVar()
today3 = StringVar()
today4 = StringVar()
today5 = StringVar()
today6 = StringVar()

tomo1v = StringVar()
tomo2v = StringVar()
tomo3v = StringVar()
tomo4v = StringVar()
tomo5v = StringVar()

tomo2 = StringVar()
tomo3 = StringVar()
tomo4 = StringVar()
tomo5 = StringVar()

rtod1v = StringVar()
rtod2v = StringVar()
rtod3v = StringVar()
rtod4v = StringVar()
rtod5v = StringVar()

rtod1 = StringVar()
rtod2 = StringVar()
rtod3 = StringVar()
rtod4 = StringVar()
rtod5 = StringVar()

rtom1v = StringVar()
rtom2v = StringVar()

rtom3v = StringVar()
rtom4v = StringVar()

rtom1 = StringVar()
rtom2 = StringVar()
rtom3 = StringVar()
rtom4 = StringVar()

g_obj = TheGUI(root)